In [1]:
# import pandas as pd
# from sklearn.datasets import load_breast_cancer

# # Load the cancer dataset
# cancer = load_breast_cancer()
# df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
# df.columns = df.columns.str.replace(" ", "_")
# df["target"] = cancer.target
# df.to_parquet("cancer.parquet")
# df.head()

import duckdb
import numpy as np

with duckdb.connect("cancer.db", read_only=False) as con:
    con.execute(
        "CREATE OR REPLACE TABLE cancer AS SELECT * FROM read_parquet('cancer.parquet')"
    )

    df = con.execute("SELECT * FROM cancer").df()


df["cv"] = np.random.randint(0, 6, df.shape[0])
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,5
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,2
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,3


In [2]:
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=2)

for col in df.columns.tolist():
    if col not in ["target", "cv"]:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

X = df.drop(columns=["target", "cv"])
y = df["target"]
X_pca = pca.fit_transform(X)

df_pca = pd.DataFrame(X_pca, columns=["pca1", "pca2"])
df_pca["target"] = y
df_pca["cv"] = df["cv"]
df_pca.head()

,pca1,pca2,target,cv
0,9.184755,1.946870,0,5
1,2.385703,-3.764859,0,1
2,5.728855,-1.074229,0,2
3,7.116691,10.266556,0,0
4,3.931842,-1.946359,0,3


In [3]:
from glam.src.data.default_model_data import DefaultModelData

dat = DefaultModelData(df_pca, y="target", cv="cv")
dat.df.head()

,pca1,pca2,target,cv
0,9.184755,1.946870,0,5
1,2.385703,-3.764859,0,1
2,5.728855,-1.074229,0,2
3,7.116691,10.266556,0,0
4,3.931842,-1.946359,0,3


In [4]:
formula = "target ~ " + " + ".join(dat.df.columns[:-2])
formula

'target ~ pca1 + pca2'

In [5]:
from glam.src.fitters import StatsmodelsFormulaGlmFitter

fitter = StatsmodelsFormulaGlmFitter(dat)
fitter.fit(formula)

fitted = fitter.fitted_model
print(fitted.coefficients)
pd.concat([fitted.yhat(), fitted.yhat_proba()], axis=1).head()

{'pca1': -2.19689615197717, 'pca2': 1.2255310106600121}


,yhat,yhat_proba
0,0.0,2.871348e-08
1,0.0,8.035403e-05
2,0.0,1.404123e-06
3,0.0,6.744946e-02
4,0.0,2.498854e-05


In [6]:
from glam.src.data.data_prep import TimeSeriesDataSplitter

ts = TimeSeriesDataSplitter(dat)
for x, y, x1, y1 in ts.X_y_generator:
    print((x.shape, y.shape, x1.shape, y1.shape))

((106, 2), (106,), (78, 2), (78,))
((184, 2), (184,), (76, 2), (76,))
((260, 2), (260,), (113, 2), (113,))
((373, 2), (373,), (87, 2), (87,))
((460, 2), (460,), (109, 2), (109,))


In [7]:
from glam.src.data.data_prep import DefaultPreprocessor

pre = DefaultPreprocessor(dat)
pre.run()

,pca1,pca2,target,cv
0,9.184755,1.946870,0,5
1,2.385703,-3.764859,0,1
2,5.728855,-1.074229,0,2
3,7.116691,10.266556,0,0
4,3.931842,-1.946359,0,3
...,...,...,...,...
564,6.433655,-3.573673,0,5
565,3.790048,-3.580897,0,0
566,1.255075,-1.900624,0,3
567,10.365673,1.670540,0,3


An issue with passing the model object around. I think there is a circular reference in the model object. I will try to fix it after lunch.

In [9]:
from glam import BinaryGlmAnalysis

analysis = BinaryGlmAnalysis(dat)

TypeError: StatsmodelsFittedGlm.__init__() missing 1 required positional argument: 'model'